In [ ]:
import pandas as pd

In [ ]:
data = pd.read_pickle("/Users/znamenp/data/cell_barcode_df.pkl")
# filter out rows where all_barcodes is NaN
data = data.dropna(subset=["all_barcodes"])

In [ ]:
data = pd.read_pickle(
    "/nemo/project/proj-znamenp-barseq/processed/becalia_rabies_barseq/BRAC8498.3e/analysis/cell_barcode_df.pkl"
)
data = data.dropna(subset=["all_barcodes"])

In [ ]:
data[data["is_starter"] == True]["all_barcodes"]

# contatenate all barcodes into a big list
all_barcodes = []
for barcodes in data[data["is_starter"] == True]["all_barcodes"]:
    all_barcodes.extend(barcodes)

# count the number of times each barcode appears among starter cells
barcode_counts = pd.Series(all_barcodes).value_counts()
# barcodes which only appear once
singletons = barcode_counts.index[barcode_counts == 1]
singletons = set(singletons.values)

In [ ]:
# select "all_barcodes" which are in signletons and assign to a new column
data["unique_barcodes"] = data["all_barcodes"].apply(
    lambda x: singletons.intersection(x)
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data[data["is_starter"] == True].apply(
    lambda x: len(x["unique_barcodes"]), axis=1
).value_counts()

# plot the distribution of the number of unique barcodes per starter cell
data[data["is_starter"] == True].apply(lambda x: len(x["all_barcodes"]), axis=1).hist(
    bins=np.arange(0, 5, 1) - 0.5, alpha=0.5
)
data[data["is_starter"] == True].apply(
    lambda x: len(x["unique_barcodes"]), axis=1
).hist(bins=np.arange(0, 5, 1) - 0.5, alpha=0.5)

plt.legend(["All barcodes", "Unique barcodes"])
# turn off the grid
import matplotlib.pyplot as plt

plt.grid(False)
# skip non-integer ticks
plt.xticks(np.arange(0, 5, 1))
plt.xlabel("Number of barcodes per starter cell")
plt.ylabel("Number of cells")

In [ ]:
# sets of unique barcodes with more than one element
doublets = (
    data[data["is_starter"] == True]
    .apply(
        lambda x: x["unique_barcodes"] if len(x["unique_barcodes"]) == 2 else None,
        axis=1,
    )
    .dropna()
    .values
)

In [ ]:
doublets

In [ ]:
bc_counts = []

for doublet in doublets:
    print(doublet)
    bc1, bc2 = doublet
    nbc1 = data.apply(lambda x: bc1 in x["unique_barcodes"], axis=1).sum()
    nbc2 = data.apply(lambda x: bc2 in x["unique_barcodes"], axis=1).sum()
    nboth = data.apply(
        lambda x: bc1 in x["unique_barcodes"] and bc2 in x["unique_barcodes"], axis=1
    ).sum()
    nany = data.apply(
        lambda x: bc1 in x["unique_barcodes"] or bc2 in x["unique_barcodes"], axis=1
    ).sum()
    print(nbc1, nbc2, nboth, nany)
    bc_counts.append([nbc1 - 1, nbc2 - 1, nboth - 1, nany - 1])

In [ ]:
np.array(bc_counts)[:, 0]

In [ ]:
plt.loglog(1 + np.array(bc_counts)[:, 0], 1 + np.array(bc_counts)[:, 1], "o")
plt.xlabel("Number of cells with barcode 1")
plt.ylabel("Number of cells with barcode 2")

In [ ]:
bc_counts = np.array(bc_counts)

plt.hist(bc_counts[:, 2] / bc_counts[:, 3])
plt.xlabel("Proportion of presynaptic cells with both barcodes")